In [ ]:
# load dependencies
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2 as cv

In [ ]:
def read(image_path, label_path):
    image = cv.imread(image_path)
    image = cv.resize(image, (640, 640))
    image /= 255.0

    label = np.zeros((80, 80, 18))
    with open(label_path, 'r') as f:
        for line in f:
            line = line.split()
            if line[0] != 0:
                continue
            x, y, w, h = float(line[1]), float(line[2]), float(line[3]), float(line[4])
            if label[int(y * 80)][int(x * 80)][0] == 0:
                label[int(y * 80)][int(x * 80)][0:6] = [1, x, y, w, h, 1]
            elif label[int(y * 80)][int(x * 80)][6] == 0:
                label[int(y * 80)][int(x * 80)][6:12] = [1, x, y, w, h, 1]
            elif label[int(y * 80)][int(x * 80)][12] == 0:
                label[int(y * 80)][int(x * 80)][12:18] = [1, x, y, w, h, 1]
            else:
                continue

    return image, label

In [ ]:
class My_Custom_Generator(tf.keras.utils.Sequence):
    def __init__(self, images, labels, batch_size):
        self.images = images
        self.labels = labels
        self.batch_size = batch_size

    def __len__(self):
        return (np.ceil(len(self.images) / float(self.batch_size))).astype(np.int)
    
    def __getitem__(self, idx):
        batch_x = self.images[idx * self.batch_size : (idx+1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]
        
        train_image = []
        train_label = []

        for i in range(len(batch_x)):
            img_path = batch_x[i]
            label = batch_y[i]
            image, label = read(img_path, label)
            train_image.append(image)
            train_label.append(label)
        return np.array(batch_x), np.array(batch_y)

In [ ]:
num_classes = 1
input_shape = (640,640,3)

def identity_block(X, f, filters):
    F1, F2 = filters
    X_shortcut = X
    X = tf.keras.layers.Conv2D(F1, (1,1), strides=(1,1), padding = 'same')(X)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.LeakyReLU(0.1)(X)
    X = tf.keras.layers.Conv2D(F2, (f,f), strides=(1,1), padding = 'same')(X)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.LeakyReLU(0.1)(X)
    X = tf.keras.layers.Add()([X, X_shortcut])
    X = tf.keras.layers.LeakyReLU(0.1)(X)
    return X

def custom_cnn(input_shape = input_shape, num_classes = num_classes):
    X_input = tf.keras.layers.Input(shape=input_shape)
    X = tf.keras.layers.Conv2D(32, (3,3), padding = 'same')(X_input)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.LeakyReLU(0.1)(X)
    X = tf.keras.layers.Conv2D(64, (3,3), strides=2, padding = 'same')(X)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.LeakyReLU(0.1)(X)
    X = identity_block(X, 3, (32,64))
    X = tf.keras.layers.Dropout(0.5)(X)
    X = tf.keras.layers.Conv2D(128, (3,3), strides=2, padding = 'same')(X)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.LeakyReLU(0.1)(X)
    X = identity_block(X, 3, (64,128))
    X = identity_block(X, 3, (64,128))
    X = tf.keras.layers.Conv2D(256, (3,3), strides=2, padding = 'same')(X)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.LeakyReLU(0.1)(X)
    X = identity_block(X, 3, (128,256))
    X = identity_block(X, 3, (128,256))
    X = identity_block(X, 3, (128,256))
    X = identity_block(X, 3, (128,256))
    X = identity_block(X, 3, (128,256))
    X = identity_block(X, 3, (128,256))
    X = identity_block(X, 3, (128,256))
    X = identity_block(X, 3, (128,256))
    X = tf.keras.layers.Conv2D(128, (1,1), padding = 'same')(X)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.LeakyReLU(0.1)(X)
    X = tf.keras.layers.Conv2D(256, (3,3), padding = 'same')(X)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.LeakyReLU(0.1)(X)
    X = tf.keras.layers.Conv2D(18, (1,1), padding = 'same')(X)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.LeakyReLU(0.1)(X)
    X = tf.keras.models.Model(X_input, X)
    return X

custom_model = custom_cnn()
custom_model.summary()